In [4]:
import numpy as np
from scipy.optimize import curve_fit
from numpy import sqrt
import pandas as pd

In [5]:
class DistributionModel:
    def __init__(self):
        ...
        
    def carbon_number(self,z,A,B):
        """Carbon number fraction function 

        Estimates the carbon number based on a composition
        and two fit  parameters
        Use the Pedersen's ratio 
        This means that it applies from carbon 6 onward
      
        Parameters
        ----------
        z: float
          Molar fraction of component
        A, B: float
            fit parameters

        Returns 
        -------
        carbon_number: float 
            number of single carbon number 
    
        """
        if A is None and B is None:
            A,B = self.A, self.B
        
        return A + B * np.log(z)

    def molar_fraction(self, carbon_number, A, B):
        """ Molar fraction function

        Estimates the molar fraction based on carbon number
        and two fit  parameters
        Use the Pedersen's ratio 
        This means that it applies from carbon 6 onwards
        
        Parameters
        ----------
        carbon_number: float
            Carbon number function
        A, B: float
            fit parameters

        Returns 
        -------
        molar_fraction: float 
            Molar fraction of a given carbon number fraction 
        """
        if A is None and B is None:
            A,B = self.A, self.B
        
        return np.exp(A+B*carbon_number)

In [6]:
class PedersenModel(DistributionModel):
    def __init__(self):
        ...
    def molar_fraction(self, carbon_number, A, B):
        """Molar fraction function
        
        Estimates the molar fraction based on carbon number
        and two fit parameters
        Use the Pedersen's ratio
        This means that it applies from carbon 6 onwards
        
        Parameters
        ----------
        carbon_number: float
            Carbon number function
        A,B: float
            fit parameters

        Returns
        -------
        molar_fraction: float
            Molar fraction of a given carbon number fraction
        """
        if A is None and B is None:
            A,B = self.A, self.B
      
        return np.exp(A+B*carbon_number)

    def molecular_weight(self, carbon_number):
        """ Molecular weight function

        Estimates the molecular weight based on a 
        carbon number fraction

        Parameters
        ----------
        carbon_number: float
            Carbon number function

        Returns 
        -------
        molecular_weight: float 
            Molecular weight of a given carbon number fraction 

        """
        return carbon_number*14-4
    
    def density(self, carbon_number, L, D):
        """ Densities function

        Estimates the densities from C6 onwards 
        based on increase with carbon number

        Parameters
        ----------
        carbon_number: float
            Carbon number function
        L, D: float
            fit parameters

        Returns 
        -------
        density: float 
            Density of a given carbon number fraction

        """
        if L is None and D is None:
            L = self.L
            D = self.D

        return L + D*np.log(carbon_number)


In [7]:
class CismondiModel(DistributionModel):
    def __init__(self):
        ...
    
    def molar_fraction(self, carbon_number, Ac, Bc):
        """ Molar fraction function

        Estimates the molar fraction based on carbon number
        and two fit  parameters
        
        Parameters
        ----------
        carbon_number: float
          Carbon number function
        Ac, Bc: float
          fit parameters

        Returns 
        -------
        molar_fraction: float 
          Molar fraction of a given carbon number fraction 
        """

    def molecular_weight(self,carbon_number,C):
    
        """ Molecular weight function

        Estimates the molecular weight based on carbon number
        and the third fit parameter of Cismondi  "C"

        Parameters
        ----------
        carbon_number: float
            Carbon number function
        c: float
            fit parameter (third parameter of cismondi et al.)

        Returns 
        -------
        molecular_weight: float 
            Molecular weight of a given carbon number fraction
        """

        if C is None:
            C = self.C

        return 84 + C*(carbon_number-6)

    def density(self,carbon_number,Ad):
        """ Density function
        Estimates the densities from C6 onwards 
        based on [...]
      
        Parameters 
        ----------
        carbon_number: float
            Carbon number function
        Ad: float
            fit parameters of cismondi's propose

        Returns
        -------
        density: float
            Density of a given carbon number fraction
        """

        if Ad is None:
            Ad = self.Ad

        Bd = 0.685 - Ad*np.exp(-0.6)
        return Ad*(np.exp(-carbon_number/10))+Bd

In [8]:
class General_Fit(DistributionModel):
    def __init__(self):
        ...
    def fit (self, x, y):
        """ Parameter setting function

        Parameters
        ----------
        x: float
            carbon number
        y: float
            molar fraction

        Return
        ------
        A,B: float
            fit parameters
        """
        self.A, self.B = curve_fit(self.carbon_number, x,y)[0]

        return 'A: ', self.A,'B: ', self.B
        
class Pedersen_Fit(PedersenModel):
    def __init__(self):
        ...
    def fit(self,x):
        """ Parameter setting function

        Parameters
        ----------
        x: float
            carbon number

        Return
        ------
        L,M: float
            fit parameters
        """ 
        self.L, self.M = curve_fit(self.density, x)[0]
        return 'L: ', self.L, 'M: ', self.M

class Cismondi_Fit(CismondiModel):
    def __init__(self):
        ...
    def fit(self,x, *args):
        y = args
        """ Parameter setting function

        Parameters
        ----------
        x: float
            carbon number 

        y: float
            molar mass for C
            density for Ad
            molar fraction for Ac, Bc
        
        Returns
        -------
        C: float
            Third parameter of Cismondi's model
        Ac, Ad: float
            fit parameter
        """ 
        self.C = curve_fit(self.molecular_weight,x,y)[0]
        self.Ac, self.Bc = curve_fit(self.molar_fraction, x,y)[0]
        self.Ad = curve_fit(self.density,x,y)[0]

        return 'C: ', self.C, 'Ac: ', self.Ac, 'Ad: ', self.Ad

In [29]:
class Correlations:
    def __init__(self,EoS):
        """ Coefficients data set to SRK or PR 

        Parameters
        ----------
        EoS: str
            Equation of state --> agregar línea para pasar el input a mayúscula para evitar errores

        Return
        ------
        c1,c2,c3,c4: float
            fit parameters to critical temperature
        d1,d2,d3,d4,d5: float
            fit parameters to critical presion
        e1,e2,e3,e4: float
            fit parameters to m_factor
        """
        coefficients = pd.read_csv('Coefficients.csv').set_index('Coefficient')
        print(coefficients)

        self.c1 = coefficients.iloc[EoS].index('c1')
        self.c2 = coefficients.iloc[EoS].index('c2')
        self.c3 = coefficients.iloc[EoS].index('c3')
        self.c4 = coefficients.iloc[EoS].index('c4')

        self.d1 = coefficients.iloc[EoS].index('d1')
        self.d2 = coefficients.iloc[EoS].index('d2')
        self.d3 = coefficients.iloc[EoS].index('d3')
        self.d4 = coefficients.iloc[EoS].index('d4')
        self.d5 = coefficients.iloc[EoS].index('d5')

        self.e1 = coefficients.iloc[EoS].index('e1')
        self.e2 = coefficients.iloc[EoS].index('e2')
        self.e3 = coefficients.iloc[EoS].index('e3')
        self.e4 = coefficients.iloc[EoS].index('e4')
        return self.c1

    def critical_temperature(self, molecular_weight,density,c1,c2,c3,c4):
        """Critical temperature correlation
        
        Parameters
        ----------
        molecular_weight: float
            Molecular weight function
        density: float
            Pedersen density
        c1,c2,c3,c4: float
            fit parameters

        Return
        ------
        critical_temperature: float
            Critical temperature
        """
        if c1 is None or c2 is None or c3 is None or c4 is None:
            c1 = self.c1
            c2 = self.c2
            c3 = self.c3
            c4 = self.c4
        
        return c1*density+c2*(np.log(molecular_weight))+c3*molecular_weight*(c4/molecular_weight)

    def critical_pression(self, molecular_weight, density, d1, d2, d3, d4, d5):
        """Critical pression correlation
        
        Parameters
        ----------
        molecular_weight: float
            Molecular weight function
        d1,d2,d3,d4,d5: float
            fit parameters

        Return
        ------
        critical_pression: float
            Critical pression
        """
        if {d1,d2,d3,d4,d5} is None:
            d1 = self.d1
            d2 = self.d2
            d3 = self.d3
            d4 = self.d4
            d5 = self.d5

        pression_log = d1 + d2*(density*(np.exp(d5)+(d3/molecular_weight)+(d4/(molecular_weight)*np.exp(2))))
    
        return np.exp(pression_log)
    
    def m_factor(self, molecular_weight, density, e1, e2, e3, e4):
        """ *m* factor (links to acentric factor as appropiate)
        
        Parameters
        ----------
        molecular_weight: float
            Molecular weight function
        e1,e2,e3,e4: float
            fit parameters

        Return
        ------
        m_factor: float
            m factor to calculate the acentric factor
        """
        if {e1,e2,e3,e4} is None:
            e1 = self.e1
            e2 = self.e2
            e3 = self.e3
            e4 = self.e4
        
        return e1+e2*molecular_weight+e3*density+e4*(molecular_weight*(np.exp(2)))

    def accentric_factor(self,m_factor):
        """ Accentric factor function
        
        Parameters
        ----------
        m_factor: float
            m factor --> links to acentric factor as appropiate
        e1,e2,e3,e4: float
            fit parameters

        Return
        ------
        accentric_factor: float
            Accentric factor
        """
        A = float(0.37464-m_factor)
        B = float(1.54226)
        C = float(-0.26992)

        ac_factor = (-B - sqrt(B*np.exp(2)-4*A*C))/(2*A)

        return ac_factor

In [30]:
Correlations(EoS='PR')

                     SRK           PR
Coefficient                          
c1            163.120000    73.404300
c2             86.052000    97.352000
c3              0.433750     0.618744
c4          -1877.400000 -2059.320000
d1             -0.134080     0.072846
d2              2.501900     2.188110
d3            208.460000   163.910000
d4          -3987.200000 -4043.230000
d5              2.000000    25.000000
e1              0.743100     0.373765
e2              0.004812     0.005493
e3              0.009671     0.001179
e4             -0.000004    -0.000005


TypeError: Cannot index by location index with a non-integer key

In [33]:
def properties_plus(self, molarfraction_values, molecularweight_values, i):
        """Function to calculate residual fraction properties
        
        Parameters
        ----------
        i: int
            Carbon number into range 
        molarfraction_values: float
            Molar fraction values to i
        molecularweight_values: float
            Molecular weight values to i

        Return
        ------
        molecularplus: float
            Molecular values to residual fraction
        molarplus: float
            Molar fraction to residual fraction
        """
        molecularplus = (molarfraction_values[:i]*molecularweight_values[:i]).sum()/(molarfraction_values[:i].sum())
        molarplus = molarfraction_values[:i].sum()

        return molecularplus, molarplus

In [34]:
class Residual_fraction(CismondiModel):
    def __init__(self,mw_max,mf_max):
        self.molecularweight_max = mw_max
        self.molarfraction_max = mf_max

    def carbon_number_max(self, carbon_range, select_function):
        """Maximum carbon number based on Cismondi's observations
        
        Parameters
        ----------
        carbon_range: int
            Carbon range to distribute
        select_function: function 
            Distribution function

        Returns
        -------
        carbonnumber_max: int
            Maximun carbon number
        """
        molarfraction_values = self.molar_fraction(carbon_range,self.Ac,self.Bc)
        molecularweight_values = self.molecular_weight(carbon_range,self.C)
        carbonnumber_max = carbon_range[0]-1

        for i in carbon_range:
            molecularplus, molarfractionplus = properties_plus(molarfraction_values,molecularweight_values,i)
            carbonnumber_max +=1
            if molecularplus > self.molecularweight_max or molarfractionplus > self.molarfraction_max:
                break 
        
        return carbonnumber_max     


In [29]:
import pandas as pd
import numpy as np

!curl "https://raw.githubusercontent.com/MarlenGabrich/GTF/main/com-fede" > com.csv
df = pd.read_csv('com.csv').drop(range(10))
df

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   667  100   667    0     0   1018      0 --:--:-- --:--:-- --:--:--  1018


,component,molar_percentage,molar_mass,density
10,C7,2.87,96,0.738
11,C8,4.08,107,0.765
12,C9,3.51,121,0.781
13,C10,3.26,134,0.792
14,C11,2.51,147,0.796
15,C12,2.24,161,0.810
16,C13,2.18,175,0.825
17,C14,2.07,190,0.836
18,C15,2.03,206,0.842
19,C16,1.67,222,0.849
